# A little theory on kNN

## Intuition

The algorithm $k$ Nearest Neighbours is one of the simpler machine learning algorithms. It is motivated by the idea that similar examples $x_t$ should have similar targets $y_t$.

So, to predict the target class of a test examples $x$, all we need to do is find the $k$ nearest neighbours to $x$ using some metric - for example, euclidean distance also known as $L_2$ norm, or more generally minkowski distance $L_p$.

In a classification problem, we use those $k$ nearest neighbours to predict the target class of $x$ as the most common target of its neighbours i.e. it's as though each neighbour of $x$ casts a vote for their own target class and the class with the most votes wins.

## Mathematic Formalism

Let
* $x$ be test examples
* $m$ be the number of classes
* $D_n = \{(x_i,y_i)\}_{i=1}^n$ be the training data where $y_i \in Y=\{1,\dots,m\}$ is the corresponding target class of example $x_i$
* $d(\dot{},\dot{})$ be our distance metric
* $V(x, k)$ is the set of $k$ nearest neighbours of $x$

<br/>
The prediction by the $k$-NN algorithm is therefore:

> $$f(x)={arg max} \left(\frac{1}{k} \sum_{(x_i,y_i) \in V(x, k)} \mathrm{onehot}_{m}(y_i)\right)$$


To find out the $k$ nearest neighbours, we need to calculate some "distance" between the test sample $x$ and all the samples in the training set, and pick the $k$ samples with the least distance from $x$.
 
A common distance function is the euclidean distance function:
> $$d(a,b)= \sqrt{\sum_{i=1}^{dim}(a_i-b_i)^2}$$

which is a specific case of the $L_p$ norm of Minkowski (where $p = 2$)
> $$d(a,b)= \left(\sum_{i=1}^d|a_i-b_i|^p\right)^\frac{1}{p}$$

<hr/>

## Pseudocode

We define a machine learning algorithm by specifying its training procedure for some training data and how to predict the target for a test example. Given that the training procedure for $k$-NN is simply loading the training data $D_n$, we can specify how to predict the target class for the case when $k = 1$:

    def kNN_1(x):
        min = +inf # intialize the distance of the nearest neighbour
        idx = -1 # initialize the index of the nearest neighbour
        
        for i=1 to n:
            dt = d(X[i], x)
            if dt < min
                min = dt
                idx = i
                
        return Y[idx]

This runs in $O(n(k+d))$ time but you can get $O(n(log(k)+d))$ time by using a priority queue (heap)

# Putting it in practice!

## Introduction

We want to make a machine learning algorithm to identify flowers. We have three types of iris species and we will try to use the characteristics of each flower (features) to determine which species of iris it is (class). But you don't know anything about flowers! So we will learn this algorithm using a dataset of flower measurements and the classes those flowers correspond to (training data), and we will use 1-kNN!

## 1. Calculate the $L^p$ (Minkowski) distance between two vectors

We want a function that given two vectors (`np.array`) will output the Minkowski distance between them. Complete the function `minkowski_vec` below. Test it yourself on two vectors (you can import the iris dataset as we did in the tutorial to use real iris vectors).

### ***Ex: Calculate Minkowski distance between two vectors***. Here is the [definition](http://en.wikipedia.org/wiki/Minkowski_distance) in case you need it

In [1]:
import numpy as np
def minkowski_vec(x1, x2, p=2.0):
    s=np.sum(np.abs(x1-x2)**2)
    dist = s**(1/2)   
    return dist

# for testing
a = np.ones((4,))
b = np.zeros((4,))
print(minkowski_vec(a, b, p=2.0))
print(minkowski_vec(a, a, p=2.0))
print (a)
print(b)

2.0
0.0
[1. 1. 1. 1.]
[0. 0. 0. 0.]


**Note:** since this is a vector, we'll need to apply operations to the different dimensions. We could do this by iterating over the array one element at a time (e.g. here to calculate the difference in absolute values)

    s = 0
    for i in range(x1.shape[0]):
        s = s + abs(x1[i] - x2[i])

or we could use `numpy` intelligently to do the same thing:

    s = np.sum(np.abs(x1 - x2))

the difference is that the second option is not just more compact and easy to read, it uses `numpy`'s library to calculate the sums and operations which is much much faster than Python because they are specialized math functions written in C++. 

In short, use numpy functions instead of for loops where possible!

## 2. Calculate $L^p$ distance between a vector and a matrix 

We also need a function to compare one flower to a bunch of other flowers. We will modify the function `minkowski_mat` to calculate an $L_p$ distance between a single vector (or 1D `np.array`) and a matrix (or 2D `np.array`). This function should return an array of distances corresponding to the distance between the single flower $x1$ and each other flower in the bunch represented as rows in $X2$.  Use `np.newaxis` or broadcasting rules.

### ***Ex: Calculate Minkowski distance between a vector and a matrix***

In [2]:
def minkowski_mat(x1, X2, p=2.0):
    dist=np.sum(np.abs(np.array(X2, dtype=np.int64)-x1)**p, axis=1)**(1/p)
    return dist


# for testing
a = np.ones((4,))
b = np.random.randint(5, size=(10,4))
print(minkowski_mat(a,b))

[4.79583152 3.74165739 1.41421356 3.74165739 4.79583152 3.60555128
 2.23606798 3.74165739 3.60555128 1.73205081]


Reiterating the very important point **the vast majority of vector-vector, vector-matrix, or matrix-matrix operations are going to be much more efficient in numpy than in python using a for loop**. 

You may have noticed that the difference in the efficient example implementations of `minkowski_vec` and `minkowski_mat` is just the part: `axis=1`. This exercise is to understand why it is important (and necessary) to specify on which axis you are applying the sum.

## 3. 1-KNN / 1-PPV

We're nearly there!

### ***Ex: Finish the following function to predict the species of a test data point with features `x`:***
- From `data`, slice the `features` and `target`
- Calculate the distance between `x` and the `features` using `minkowski_mat`
- Find the data point with the least distance, and return its `target`

In [3]:
def knn(x, data, p=2):
    features=np.delete(data, 0, 1)
    x=np.array(x).reshape(1, len(features[0]))
    l = list()
    l = minkowski_mat(x, features, p=2)
    return(min(l))

Note that `x` is the array of features (no target) of the test example. This functions should be quite easy to write then because `minkowski_mat` will output an array of distances, and we want to find the *index* of the *minimum* of those distances, then just find the target at that index.

## Conclusion

We now have all the components of the 1-NN algorithm, all that's left is to put it all together and test it

Remember that we can access the functions we've executed in the previous cells

After testing your implementation, write a `for` loop which will call `knn(iris[i,:-1], iris, p)` for each example `i` and compare the prediction to the true target `iris[i,-1]`. The two should always be the same, why?

In [4]:
import numpy as np
iris = np.loadtxt('iris.txt')

predictions = np.zeros(iris.shape[0])
for i in range(iris.shape[0]):
    predictions[i] = knn(iris[i,:-1],iris)
    
targets = iris[:,-1]
print("error rate: {} %".format(100-(np.abs(predictions-targets)).mean()))

error rate: 97.68960687936965 %


## Bonus and things to reflect on for the next time

In machine learning, we usually have a split of training data and testing data
* divide the dataset in two : one training dataset with 50 examples (randomly sampled) and a testing dataset with the remaining examples. 
* Using the training data to find the nearest neighbours and target output for a given example, calculate the performance of your algorithm on training and testing. Why is there such a difference?

In [5]:
indexes = np.arange(iris.shape[0])
# set the random seed so we have exact reproducibility
np.random.seed(3395)
np.random.shuffle(indexes)

train_set = iris[indexes[:50]]
test_set = iris[indexes[50:]]

# predictions on the training set
train_predictions = np.zeros(train_set.shape[0])
for i in range(train_set.shape[0]):
    train_predictions[i] = knn(train_set[i,:-1],train_set)
# predictions on the testing set
test_predictions = np.zeros(test_set.shape[0])
for i in range(test_set.shape[0]):
    test_predictions[i] = knn(test_set[i,:-1],train_set)
print("Training data error rate: {} %".format(100-(np.abs(train_predictions-train_set[:,-1])).mean()))
print("Testing data error rate: {} %".format(100-(np.abs(test_predictions-test_set[:,-1])).mean()))

Training data error rate: 97.79073790543801 %
Testing data error rate: 97.62842371414011 %
